In [1]:
import datasets
import utils
import gmm_impute
import gmm_dae
import pickle_utils as pu
import numpy as np
import pandas as pd
import itertools as it

from sklearn.mixture import BayesianGaussianMixture

import imp
imp.reload(utils)
imp.reload(datasets)
imp.reload(gmm_impute)
imp.reload(gmm_dae)

<module 'gmm_dae' from '/home/adria/MIMIC/ts-baselines/timeless-imputation/gmm_dae.py'>

In [18]:
imp.reload(datasets)
dsets = datasets.datasets()
baseline = datasets.benchmark({'MICE': datasets.memoize(utils.impute_mice),
                               'MissForest': datasets.memoize(utils.impute_missforest)}, dsets)

+++ Importing Ionosphere
V2 ; must have more than 1 possible value
+++ Importing Soybean
+++ Importing BreastCancer
+++ Importing Servo
+++ Importing BostonHousing


In [19]:
baseline.loc['mean_std']

MICE                                 \
                                 NRMSE       PFC      RMSE total_cats   
BostonHousing MCAR_rows  0.1  0.198712  0.062500  0.141961       48.0   
                         0.3  0.280159  0.062500  0.176939      160.0   
                         0.5  0.290166  0.091633  0.198751      251.0   
                         0.7  0.351093  0.075676  0.224232      370.0   
                         0.9  0.394617  0.074890  0.253997      454.0   
              MCAR_total 0.1  0.216072  0.068182  0.138375       44.0   
                         0.3  0.233771  0.100000  0.153573      160.0   
                         0.5  0.244531  0.065385  0.168912      260.0   
                         0.7  0.300698  0.077562  0.201955      361.0   
                         0.9       NaN  0.072650       NaN      468.0   
BreastCancer  MCAR_rows  0.1       NaN  0.469551       NaN      624.0   
                         0.3       NaN  0.455298       NaN     1812.0   
                         0.5       NaN  0.461824       NaN     3235.0   
                         0.7       NaN  0.511278       NaN     4389.0   
                         0.9       NaN  0.631768       NaN     5559.0   
              MCAR_total 0.1       NaN  0.473684       NaN      608.0   
                         0.3       NaN  0.454358       NaN     1939.0   
                         0.5       NaN  0.462433       NaN     3181.0   
                         0.7       NaN  0.497309       NaN     4460.0   
                         0.9       NaN  0.670976       NaN     5647.0   
Ionosphere    MCAR_rows  0.1  0.794563  0.121212  0.224502       33.0   
                         0.3  0.774686  0.125000  0.221319       96.0   
                         0.5  0.841665  0.126437  0.249482      174.0   
                         0.7  0.892347  0.111554  0.260353      251.0   
                         0.9  1.066690  0.119874  0.309771      317.0   
              MCAR_total 0.1  0.762424  0.090909  0.224541       33.0   
                         0.3  0.784978  0.075630  0.225260      119.0   
                         0.5  0.789150  0.110526  0.230501      190.0   
                         0.7  0.844955  0.104803  0.244828      229.0   
                         0.9       NaN  1.000000       NaN      315.0   
Servo         MCAR_rows  0.1       NaN  0.661538       NaN       65.0   
                         0.3       NaN  0.735450       NaN      189.0   
                         0.5       NaN  0.731481       NaN      324.0   
                         0.7       NaN  0.746237       NaN      465.0   
                         0.9       NaN  0.750825       NaN      606.0   
              MCAR_total 0.1       NaN  0.629032       NaN       62.0   
                         0.3       NaN  0.670000       NaN      200.0   
                         0.5       NaN  0.700935       NaN      321.0   
                         0.7       NaN  0.718280       NaN      465.0   
                         0.9       NaN  0.741294       NaN      603.0   
Soybean       MCAR_rows  0.1       NaN  0.186561       NaN     2262.0   
                         0.3       NaN  0.213844       NaN     6140.0   
                         0.5       NaN  0.272023       NaN    10591.0   
                         0.7       NaN  0.312789       NaN    14927.0   
                         0.9       NaN  0.522232       NaN    19679.0   
              MCAR_total 0.1       NaN  0.165007       NaN     2109.0   
                         0.3       NaN  0.185701       NaN     6392.0   
                         0.5       NaN  0.225235       NaN    10731.0   
                         0.7       NaN  0.277249       NaN    15138.0   
                         0.9       NaN  0.671050       NaN    19316.0   

                             MissForest                                 
                                  NRMSE       PFC      RMSE total_cats  
BostonHousing MCAR_rows  0.1   0.167288  0.041667  0.101091       48.0  
                  

In [4]:
BGMMs = {}
for name, df in dsets.items():
    m = BayesianGaussianMixture(n_components=100, covariance_type='full', max_iter=10000)
    m.fit(df.values)
    BGMMs[len(df)] = gmm_impute.prune_infinite_gmm(df.values, m)

In [5]:
m = BGMMs[len(dsets["BostonHousing"])]
from gmm_impute import mask_matrix
mask = np.zeros([m['means'].shape[1]], dtype=np.bool)
mask[[1,2,5,7,9,10]] = True
print("mask:", mask)
inp = dsets["BostonHousing"].values[0]

def compute_classically():
    K_12 = mask_matrix(m['covariances'],mask,~mask)
    K_22__1 = np.linalg.inv(mask_matrix(m['covariances'],~mask,~mask))
    K_1222 = K_12 @ K_22__1
    K_21 = mask_matrix(m['covariances'],~mask,mask)
    K_11 = mask_matrix(m['covariances'],mask,mask)

    diff = np.expand_dims(inp[~mask] - m['means'][:,~mask], axis=2)
    return (m['means'][:,mask] + np.squeeze(K_1222 @ diff, axis=2)), (K_11 - K_1222 @ K_21)

def compute_newly():
    K12 = m['covariances']*(mask[:,None])*(~mask)

mask: [False  True  True False False  True False  True False  True  True False]


In [24]:
%%time
imp.reload(gmm_impute)
imp.reload(datasets)
def gmm_impute_f(_path, dataset, full_data):
    _dataset = dataset.values
    m = BayesianGaussianMixture(n_components=100, covariance_type='full', max_iter=10000)
    m.fit(full_data.values)
    m = gmm_impute.prune_infinite_gmm(full_data.values, m)
    ndarray = gmm_impute._gmm_impute(m, _dataset, n_impute=5)
    return list(pd.DataFrame(a, index=dataset.index, columns=dataset.columns) for a in ndarray)
benchmark_gmm = datasets.benchmark({'GMM': gmm_impute_f}, dsets)

CPU times: user 3min 53s, sys: 8min 38s, total: 12min 31s
Wall time: 1min 33s


In [7]:
#%%time
#def ff(p, d):
#    return gmm_dae.gmm_dae_impute(p, d, BGMMs[len(d)])
#gmm_dae = datasets.benchmark({'GMM_dae': ff}, dsets)

In [27]:
print("RECALL: the GMMs are learned from the full data.")
pd.concat([baseline, benchmark_gmm], axis=1)

RECALL: the GMMs are learned from the full data.


MICE           MissForest  \
                                           RMSE RMSE_best  RMSE_best   
mean_std BostonHousing MCAR_rows  0.1  1.970576  0.667243   0.451820   
                                  0.3  2.229830  0.776376   0.613673   
                                  0.5  2.388023  0.803711   0.665862   
                                  0.7  2.692048  0.931416   0.836119   
                                  0.9  3.568152  1.360333   1.319464   
                       MCAR_total 0.1  1.845577  0.633651   0.442666   
                                  0.3  2.056155  0.727090   0.540693   
                                  0.5  2.256642  0.778487   0.648135   
                                  0.7  2.434157  0.849971   0.832745   
                                  0.9       NaN       NaN   0.980250   
         Ionosphere    MCAR_rows  0.1  2.322066  0.844523   0.671896   
                                  0.3  2.449642  0.872998   0.731530   
                                  0.5  2.668912  0.967708   0.804740   
                                  0.7  2.681691  0.941558   0.822890   
                                  0.9  3.580123  1.151242   0.984018   
                       MCAR_total 0.1  2.256829  0.850685   0.688199   
                                  0.3  2.370199  0.911685   0.742551   
                                  0.5  2.432654  0.889776   0.779419   
                                  0.7  2.589056  0.914817   0.829445   
                                  0.9       NaN       NaN   1.137446   
min_max  BostonHousing MCAR_rows  0.1  2.200566  0.798172   0.782840   
                                  0.3  2.240450  0.782467   0.749781   
                                  0.5  2.306887  0.754829   0.789096   
                                  0.7  2.345940  0.720277   0.742187   
                                  0.9  2.471487  0.725131   0.813648   
                       MCAR_total 0.1  2.290057  0.877015   0.850275   
                                  0.3  2.228190  0.822236   0.815926   
                                  0.5  2.334629  0.823197   0.811226   
                                  0.7  2.253788  0.743867   0.823914   
                                  0.9       NaN       NaN   0.781481   
         Ionosphere    MCAR_rows  0.1  2.420046  0.897856   0.793047   
                                  0.3  2.567332  0.938198   0.824238   
                                  0.5  2.641299  0.952435   0.837228   
                                  0.7  2.464414  0.814190   0.801801   
                                  0.9  3.509586  1.106951   0.819047   
                       MCAR_total 0.1  2.509329  0.982017   0.821502   
                                  0.3  2.528828  0.993275   0.835257   
                                  0.5  2.475948  0.913348   0.835149   
                                  0.7  2.475882  0.849851   0.856101   
                                  0.9       NaN       NaN   0.932666   

                                            GMM            
                                           RMSE RMSE_best  
mean_std BostonHousing MCAR_rows  0.1  1.072804  0.365254  
                                  0.3  1.654981  0.611816  
                                  0.5  1.839119  0.647989  
                                  0.7  2.279782  0.815762  
                                  0.9  3.548104  1.283048  
                       MCAR_total 0.1  0.894993  0.334596  
                                  0.3  1.125346  0.403708  
                                  0.5  1.497538  0.536686  
                                  0.7  1.988131  0.710435  
                                  0.9  2.495571  0.903110  
         Ionosphere    MCAR_rows  0.1  0.366936  0.113734  
                                  0.3  0.636381  0.234762  
                                  0.5  0.737381  0.272866  
                                  0.7  1.339072  0.482290  
                                  0.9  2.254296  0.846429 